In [110]:
!pip install requests --upgrade --quiet

In [111]:
!pip install beautifulsoup4 --upgrade --quiet

In [112]:
import os

In [113]:
import pandas as pd

#### Create a function to return a document having all the html info parsed to it

In [85]:
import requests
from bs4 import BeautifulSoup

def get_books_page():
    # Download the page which we need to scrape
    web_url = 'http://books.toscrape.com/index.html'
    r = requests.get(web_url)
    if r.status_code != 200:
        raise Exception('Failed to load page {}'.format(web_url))
    books_doc = BeautifulSoup(r.text, 'html.parser')
    return books_doc

#### Create a Function to extract the list of all the genre of books

In [62]:
def get_book_genre(books_doc):
    tag =books_doc.find_all('ul', {'class' : 'nav nav-list'})
    li_tag = tag[0].find_all('ul')
    genre_tag = li_tag[0].find_all('a')
    genre_title = []
    for x in genre_tag:
        genre_title.append(x.text.strip())
    return genre_title

#### Create a finction to extract the URL's all of genres

In [65]:
def get_genre_url(books_doc):
    tag =books_doc.find_all('ul', {'class' : 'nav nav-list'})
    li_tag = tag[0].find_all('ul')
    genre_tag = li_tag[0].find_all('a')
    genre_url =[]
    base_url = 'http://books.toscrape.com/index.html'
    for y in genre_tag:
        genre_url.append(base_url + y['href'])
    return genre_url

#### Let put this together in a single function

In [66]:
def scrape_genre_info():
    web_url = 'http://books.toscrape.com/index.html'
    r = requests.get(web_url)
    if r.status_code != 200:
        raise Exception('Failed to load page {}'.format(web_url))
    books_doc = BeautifulSoup(r.text, 'html.parser')
    genre_dict = {
        'title': get_book_genre(books_doc),
        'url': get_genre_url(books_doc)
    }
    return pd.DataFrame(genre_dict)

In [86]:
book_doc = get_books_page()

In [148]:
title = get_book_genre(book_doc)

In [149]:
url = get_genre_url(book_doc) 

In [150]:
dataframe = scrape_genre_info()

#### Convert this information into CSV file

In [151]:
dataframe.to_csv('genre_page_info.csv',index=None)

### Getting the information of books of each genre

#### Downloading the genre pages

In [152]:
def get_genre_page(genre_url):
    # Download the page
    r = requests.get(genre_url)
    # Parse using Beautiful soup
    genre_doc = BeautifulSoup(r.text, 'html.parser')
    return genre_doc

In [153]:
def get_book_info(h3_tag, price_tag):
    # returns all the required info about a book
    a_tags = h3_tag.find_all('a')
    title = a_tags[0]['title']
    book_url =  base_url + a_tags[0]['href']
    price = price_tag.text.strip()
    return title, book_url, price

In [157]:
def get_books(genre_doc):
    h3_tag = genre_doc.find_all('h3')
    price_tag = genre_doc.find_all('p', {'class' : 'price_color'})
    
    book_info_dict = {'title': [],
                      'book_url': [],
                      'book_price': []}
    
    for l in range(len(h3_tag)):
        
        book_info = get_book_info(h3_tag[i], price_tag[i])
        
        book_info_dict['title'].append(book_info[0])
        book_info_dict['book_url'].append(book_info[1])
        book_info_dict['price'].append(book_info[2])
        
    final_dict_df = pd.DataFrame(book_info_dict)
        
    return final_dict_df

In [158]:
def scrape_book(genre_url, path):
    if os.path.exists(path):
        print("The file {} already exists. Skipping...".format(path))
        return
    genre_df = get_books(get_genre_page(genre_url))
    genre_df.to_csv(path , index=None)


In [159]:
def scrape_topics_repos():
    print('Scraping list of topics')
    genres_df = scrape_genre_info()
    
    for index,row in genres_df.iterrows():
        print('Scraping top repositories for "{}"'.format(row['title']))
        scrape_book(row['url'], row['title'])

In [ ]:
scrape_topics_repos()

Scraping list of topics
Scraping top repositories for "Travel"
Scraping top repositories for "Mystery"
Scraping top repositories for "Historical Fiction"
Scraping top repositories for "Sequential Art"
Scraping top repositories for "Classics"
Scraping top repositories for "Philosophy"
Scraping top repositories for "Romance"
